**Fake News Classifier Using LSTM**

In [45]:
import pandas as pd

In [46]:
df=pd.read_csv('/content/train.csv')

In [47]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [48]:
###Drop Nan Values
df=df.dropna()

In [49]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [50]:
## Get the Dependent features
y=df['label']

In [51]:
X.shape

(18285, 4)

In [52]:

y.shape

(18285,)

In [53]:
import tensorflow as tf

In [54]:
tf.__version__

'2.3.0'

In [55]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [56]:
### Vocabulary size
voc_size=5000

**Onehot Representation**

In [57]:

messages=X.copy()

In [58]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [60]:
messages.reset_index(inplace=True)

In [61]:
import nltk
import re
from nltk.corpus import stopwords

In [62]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [64]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [65]:

corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [66]:

onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2454, 3781, 2774, 403, 3668, 2522, 671, 4372, 3156, 3965],
 [2458, 1239, 1914, 2141, 3156, 1402, 717],
 [1293, 4567, 2466, 3984],
 [3452, 482, 3148, 17, 1417, 1323],
 [6, 3156, 599, 4567, 1046, 3527, 3156, 2198, 4174, 4301],
 [1601,
  964,
  3674,
  1477,
  2488,
  74,
  1880,
  3544,
  3736,
  3658,
  3441,
  3901,
  1770,
  3640,
  717],
 [2107, 3150, 4379, 3068, 191, 3012, 402, 297, 209, 4524, 959],
 [720, 4339, 1589, 335, 3106, 36, 74, 2835, 209, 4524, 959],
 [3951, 1469, 1832, 3246, 989, 4345, 1509, 1412, 74, 321],
 [267, 493, 2230, 4633, 3517, 228, 3026, 1917],
 [3435, 4535, 2203, 2988, 4989, 939, 2854, 4947, 2096, 1512, 724],
 [17, 3583, 3668, 4345, 74, 3106],
 [1072, 3622, 4607, 3844, 4783, 3550, 567, 3705, 2893],
 [1706, 1740, 4240, 1401, 3326, 3585, 654, 209, 4524, 959],
 [1226, 2902, 3247, 4879, 4050, 209, 4524, 959],
 [2941, 429, 2324, 1010, 3766, 1924, 2438, 2220, 1497, 4801],
 [3161, 4254, 1239],
 [199, 1259, 4645, 1840, 74, 3052, 2836, 717],
 [4088, 4913, 1914, 188, 48

**Embedding Representation**

In [67]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4372 3156 3965]
 [   0    0    0 ... 3156 1402  717]
 [   0    0    0 ... 4567 2466 3984]
 ...
 [   0    0    0 ...  209 4524  959]
 [   0    0    0 ... 4537 4272  282]
 [   0    0    0 ... 2164 2102 2504]]


In [68]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [69]:
len(embedded_docs),y.shape

(18285, (18285,))

In [70]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [71]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [72]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

**Model Training**

In [73]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 36ms/step - loss: 0.3270 - accuracy: 0.8494 - val_loss: 0.2067 - val_accuracy: 0.9153
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1358 - accuracy: 0.9477 - val_loss: 0.1927 - val_accuracy: 0.9186
Epoch 3/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0979 - accuracy: 0.9647 - val_loss: 0.2115 - val_accuracy: 0.9130
Epoch 4/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0697 - accuracy: 0.9758 - val_loss: 0.2780 - val_accuracy: 0.8998
Epoch 5/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0485 - accuracy: 0.9835 - val_loss: 0.3083 - val_accuracy: 0.9152
Epoch 6/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0313 - accuracy: 0.9906 - val_loss: 0.3009 - val_accuracy: 0.9016
Epoch 7/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0203 - accuracy: 0.9945 - val_loss: 0.4564 - val_accuracy: 0.9185

**Performance Metrics And Accuracy**

In [77]:

y_pred=model.predict_classes(X_test)

In [78]:
from sklearn.metrics import confusion_matrix

In [79]:
confusion_matrix(y_test,y_pred)

array([[3091,  328],
       [ 173, 2443]])

In [80]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9169842584921293